In [45]:
import pandas as pd
import hvplot.pandas
import requests
import json
import datetime
import time
import numpy as np
import yfinance as yf
from pycoingecko import CoinGeckoAPI
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from finta import TA
from holoviews import opts

## Prepare and Clean up Litecoin Dataset

In [2]:
# Create URL for Litecoin data
response = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=litecoin').json()
#response

In [3]:
cg = CoinGeckoAPI()

In [4]:
# Set timeframe to get the range of Litecoin data
def unix_time(year, month, day, hour, second):
    date_time = datetime.datetime(year, month, day, hour, second)
    return time.mktime(date_time.timetuple())

def human_time(unix_time):
    return datetime.datetime.fromtimestamp(unix_time)

In [5]:
# Set start time and end time in timestamp format
start_time = unix_time(2013, 12, 14, 0 ,0)
end_time = unix_time(2022, 11, 27, 0, 0)
print(f"start_time: {start_time}")
print(f"end_time: {end_time}")

start_time: 1386997200.0
end_time: 1669525200.0


In [6]:
# Get Litecoin data according to my timeframe
ltc_result = cg.get_coin_market_chart_range_by_id(
    id='litecoin',
    vs_currency='usd',
    from_timestamp=start_time,
    to_timestamp=end_time
)

In [7]:
# Create a dataframe for Litecoin
ltc_df = pd.DataFrame(ltc_result['prices'])
columns = ['Timestamp', 'Open Price']
ltc_df.columns = columns
ltc_df.head()

,Timestamp,Open Price
0,1387065600000,30.9159
1,1387152000000,26.3484
2,1387238400000,23.2584
3,1387324800000,15.8564
4,1387411200000,18.0604


In [8]:
# Convert timestamp to datetime format, drop Timestamp columns, set Date as index
ltc_df['Date'] = pd.to_datetime(ltc_df['Timestamp'], unit='ms')
ltc_df = ltc_df.drop(columns='Timestamp')
ltc_df = ltc_df.set_index('Date')
ltc_df

,Open Price
Date,
2013-12-15,30.915900
2013-12-16,26.348400
2013-12-17,23.258400
2013-12-18,15.856400
2013-12-19,18.060400
...,...
2022-11-23,70.339749
2022-11-24,79.217674
2022-11-25,78.822501


In [9]:
# Use Yahoo Finance library to get close price dataframe
ltc_yf = yf.download('LTC-USD', start='2013-12-15', end='2022-11-28', progress=False)
ltc_yf = ltc_yf.drop(columns='Adj Close')
ltc_yf

,Open,High,Low,Close,Volume
Date,,,,,
2014-09-17,5.085890,5.170770,4.965950,5.058550,3071840
2014-09-18,5.065430,5.065430,4.579960,4.685230,4569260
2014-09-19,4.687290,4.755820,4.254350,4.327770,3917450
2014-09-20,4.329200,4.616080,4.202190,4.286440,5490660
2014-09-21,4.263070,4.300130,4.154990,4.245920,2931220
...,...,...,...,...,...
2022-11-23,70.464264,81.563438,69.785896,79.000069,2115042339
2022-11-24,78.999107,79.031052,76.008377,78.830147,1248286495
2022-11-25,78.807121,78.857140,73.604179,74.028900,891650503


In [10]:
# Get close price and open price only
ltc_df['Closed Price'] = ltc_yf[['Close']].copy(). dropna()
ltc_df = ltc_df.dropna()
ltc_df.head()

,Open Price,Closed Price
Date,,
2014-09-17,5.022837,5.05855
2014-09-18,4.680537,4.68523
2014-09-19,4.316283,4.32777
2014-09-20,4.264892,4.28644
2014-09-21,4.206361,4.24592


In [11]:
# Create a column to get price change percentage, drop columns that have null value
ltc_df['Actual Returns'] = ltc_df['Closed Price'].pct_change()
ltc_df = ltc_df.dropna()
display(ltc_df.head())
display(ltc_df.tail())

C:\Users\Ivan\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open Price,Closed Price,Actual Returns
Date,,,
2014-09-18,4.680537,4.68523,-0.073800
2014-09-19,4.316283,4.32777,-0.076295
2014-09-20,4.264892,4.28644,-0.009550
2014-09-21,4.206361,4.24592,-0.009453
2014-09-22,4.213722,4.24235,-0.000841


,Open Price,Closed Price,Actual Returns
Date,,,
2022-11-23,70.339749,79.000069,0.121478
2022-11-24,79.217674,78.830147,-0.002151
2022-11-25,78.822501,74.028900,-0.060906
2022-11-26,74.035992,76.372894,0.031663
2022-11-27,76.342855,75.044319,-0.017396


## Generate trade type, trading signals using short and long window SMA values

In [12]:
# Generate the fast and slow simple moving averages
short_window = 20
long_window = 100

ltc_df['SMA_Short'] = ltc_df['Closed Price'].rolling(window=short_window).mean()
ltc_df['SMA_Long'] = ltc_df['Closed Price'].rolling(window=long_window).mean()
ltc_df = ltc_df.dropna()
ltc_df = ltc_df.drop(columns='Open Price')
display(ltc_df.head())
display(ltc_df.tail())

,Closed Price,Actual Returns,SMA_Short,SMA_Long
Date,,,,
2014-12-26,2.78169,0.030465,3.157594,3.737192
2014-12-27,2.71695,-0.023274,3.107883,3.717509
2014-12-28,2.73304,0.005922,3.061875,3.701562
2014-12-29,2.69482,-0.013984,3.020558,3.685645
2014-12-30,2.70401,0.003410,2.980765,3.670226


,Closed Price,Actual Returns,SMA_Short,SMA_Long
Date,,,,
2022-11-23,79.000069,0.121478,62.694070,56.736650
2022-11-24,78.830147,-0.002151,63.255748,56.911528
2022-11-25,74.028900,-0.060906,63.471133,57.048075
2022-11-26,76.372894,0.031663,63.885475,57.211347
2022-11-27,75.044319,-0.017396,64.255077,57.421474


In [13]:
# Initialize the new Signal column
ltc_df['Signal'] = 0.0

ltc_df.loc[(ltc_df['Actual Returns'] >= 0), 'Signal'] = 1
ltc_df.loc[(ltc_df['Actual Returns'] < 0), 'Signal'] = -1

ltc_df.head()

,Closed Price,Actual Returns,SMA_Short,SMA_Long,Signal
Date,,,,,
2014-12-26,2.78169,0.030465,3.157594,3.737192,1.0
2014-12-27,2.71695,-0.023274,3.107883,3.717509,-1.0
2014-12-28,2.73304,0.005922,3.061875,3.701562,1.0
2014-12-29,2.69482,-0.013984,3.020558,3.685645,-1.0
2014-12-30,2.70401,0.003410,2.980765,3.670226,1.0


In [14]:
# Create a new column Trade type
ltc_df['Trade_Type'] = np.nan

previous_price = 0
for index, row in ltc_df.iterrows():
    if previous_price == 0:
        ltc_df.loc[index, 'Trade_Type'] = 'buy'
    elif row['Closed Price'] < previous_price:
        ltc_df.loc[index, 'Trade_Type'] = 'buy'
    elif row['Closed Price'] > previous_price:
        ltc_df.loc[index, 'Trade_Type'] = 'sell'
    else:
        ltc_df.loc[index, 'Trade_Type'] = 'hold'
        
    previous_price = row["Closed Price"]

display(ltc_df.head())
display(ltc_df.tail())

,Closed Price,Actual Returns,SMA_Short,SMA_Long,Signal,Trade_Type
Date,,,,,,
2014-12-26,2.78169,0.030465,3.157594,3.737192,1.0,buy
2014-12-27,2.71695,-0.023274,3.107883,3.717509,-1.0,buy
2014-12-28,2.73304,0.005922,3.061875,3.701562,1.0,sell
2014-12-29,2.69482,-0.013984,3.020558,3.685645,-1.0,buy
2014-12-30,2.70401,0.003410,2.980765,3.670226,1.0,sell


,Closed Price,Actual Returns,SMA_Short,SMA_Long,Signal,Trade_Type
Date,,,,,,
2022-11-23,79.000069,0.121478,62.694070,56.736650,1.0,sell
2022-11-24,78.830147,-0.002151,63.255748,56.911528,-1.0,buy
2022-11-25,74.028900,-0.060906,63.471133,57.048075,-1.0,buy
2022-11-26,76.372894,0.031663,63.885475,57.211347,1.0,sell
2022-11-27,75.044319,-0.017396,64.255077,57.421474,-1.0,buy


In [15]:
ltc_df['Trade_Type'].value_counts()

sell    1446
buy     1445
Name: Trade_Type, dtype: int64

In [16]:
ltc_df['Signal'].value_counts()

 1.0    1447
-1.0    1444
Name: Signal, dtype: int64

## Preparation for model testing with training and testing dataset

In [17]:
# Assign SMA_Short and SMA_Long dataframe to X
X = ltc_df[['SMA_Short', 'SMA_Long']].shift().dropna()
X.head()

,SMA_Short,SMA_Long
Date,,
2014-12-27,3.157594,3.737192
2014-12-28,3.107883,3.717509
2014-12-29,3.061875,3.701562
2014-12-30,3.020558,3.685645
2014-12-31,2.980765,3.670226


In [18]:
# Assign Signal columns dataframe to y 
y = ltc_df['Signal']
y.value_counts()

 1.0    1447
-1.0    1444
Name: Signal, dtype: int64

In [19]:
# Select the start and end of traning period
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=40)
print(f"training_begin: {training_begin}")
print(f"training_end: {training_end}")

training_begin: 2014-12-27 00:00:00
training_end: 2018-04-27 00:00:00


In [20]:
# Generate the X_train and y_train Dataframes
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

display(X_train.head())
display(X_train.tail())

,SMA_Short,SMA_Long
Date,,
2014-12-27,3.157594,3.737192
2014-12-28,3.107883,3.717509
2014-12-29,3.061875,3.701562
2014-12-30,3.020558,3.685645
2014-12-31,2.980765,3.670226


,SMA_Short,SMA_Long
Date,,
2018-04-23,128.923800,170.62664
2018-04-24,129.812900,169.85550
2018-04-25,132.181350,169.14475
2018-04-26,133.463001,168.00046
2018-04-27,135.489201,167.15050


In [21]:
# Generate X_test and y_test Dataframes
X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,SMA_Short,SMA_Long
Date,,
2018-04-28,136.971451,166.28181
2018-04-29,138.702701,165.92146
2018-04-30,140.634751,165.59261
2018-05-01,142.345451,165.14901
2018-05-02,143.855302,164.70425


,SMA_Short,SMA_Long
Date,,
2022-11-23,61.840132,56.554904
2022-11-24,62.694070,56.736650
2022-11-25,63.255748,56.911528
2022-11-26,63.471133,57.048075
2022-11-27,63.885475,57.211347


In [22]:
# Create a StandardScaler instance and transfer those dataset to Z-score value
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Use AdaBoost Model to predict trading signals and strategy return

In [23]:
# Declare a AdaBoost model.
ada_model = AdaBoostClassifier(n_estimators=500, random_state=1)
ada_model = ada_model.fit(X_train_scaled, y_train)
ada_pred = ada_model.predict(X_test_scaled)
ada_pred[:10]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [24]:
# Use a classification report to evaluate the model using the predictions and testing data
ada_testing_report = classification_report(y_test, ada_pred)
print(ada_testing_report)

              precision    recall  f1-score   support

        -1.0       0.49      0.25      0.33       836
         1.0       0.50      0.74      0.59       839

    accuracy                           0.49      1675
   macro avg       0.49      0.49      0.46      1675
weighted avg       0.49      0.49      0.46      1675



In [25]:
# Create a predictions DataFrame for AdaBoost Model
ada_preditions_df = pd.DataFrame(index=X_test.index)

ada_preditions_df['Predicted'] = ada_pred
ada_preditions_df['Actual Returns'] = ltc_df['Actual Returns']
ada_preditions_df['Strategy Returns'] = ada_preditions_df['Predicted'] * ada_preditions_df['Actual Returns']

ada_preditions_df.head()

,Predicted,Actual Returns,Strategy Returns
Date,,,
2018-04-28,-1.0,0.043803,-0.043803
2018-04-29,-1.0,0.008766,-0.008766
2018-04-30,-1.0,-0.033548,0.033548
2018-05-01,-1.0,0.000074,-0.000074
2018-05-02,-1.0,0.020803,-0.020803


In [26]:
# Plot the actual returns versus the strategy returns
(1 + ada_preditions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title='AdaBoost Model - Actual Returns vs Strategy Returns')

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

## Use Neural Network to predict trading type

In [27]:
# Define y output variable
y = ltc_df['Trade_Type']
y.value_counts()

sell    1446
buy     1445
Name: Trade_Type, dtype: int64

In [28]:
# Convert trading type to integers
converter = LabelEncoder()
converter.fit(y)
convert_y = converter.transform(y)
convert_y

array([0, 0, 1, ..., 0, 1, 0])

In [29]:
# Convert labeled integers to a Keras `categorical` data type
y_categorical = to_categorical(convert_y, num_classes=2)
y_categorical

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [30]:
# Specify X (predictor) variables
X1 = ltc_df[['SMA_Short', 'SMA_Long']]
X1.head()

,SMA_Short,SMA_Long
Date,,
2014-12-26,3.157594,3.737192
2014-12-27,3.107883,3.717509
2014-12-28,3.061875,3.701562
2014-12-29,3.020558,3.685645
2014-12-30,2.980765,3.670226


In [31]:
# Split into training and testing windows
X1_train, X1_test, y_train, y_test = train_test_split(X1, y_categorical, random_state=1)

In [32]:
pd.DataFrame(y_train).sum()

0    1075.0
1    1093.0
dtype: float32

In [33]:
# Define the the number of inputs (features) to the model
number_input_features = len(X1.columns)
number_input_features

2

In [34]:
# Build the neural network layers
model = Sequential()
model.add(Dense(48, input_dim=number_input_features, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(18, activation='relu'))
#model.add(Dense(18, activation='relu'))

In [35]:
# Add the final output layer
model.add(Dense(2, activation='softmax'))

In [36]:
# Compile the model (with multi-class specific parameters)
model.compile(loss="binary_crossentropy", 
              optimizer= "adam", 
              metrics=['accuracy'])

In [37]:
# Summarise the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                144       
                                                                 
 dense_1 (Dense)             (None, 38)                1862      
                                                                 
 dense_2 (Dense)             (None, 18)                702       
                                                                 
 dense_3 (Dense)             (None, 2)                 38        
                                                                 
Total params: 2,746
Trainable params: 2,746
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Fit the model
model.fit(X1_train,y_train, epochs=120, batch_size=1000, shuffle=True)

Epoch 1/120
3/3 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.5088
Epoch 2/120
3/3 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5042
Epoch 3/120
3/3 [==============================] - 0s 3ms/step - loss: 0.6943 - accuracy: 0.4982
Epoch 4/120
3/3 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5097
Epoch 5/120
3/3 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.5101
Epoch 6/120
3/3 [==============================] - 0s 2ms/step - loss: 0.6927 - accuracy: 0.5069
Epoch 7/120
3/3 [==============================] - 0s 3ms/step - loss: 0.6935 - accuracy: 0.4977
Epoch 8/120
3/3 [==============================] - 0s 2ms/step - loss: 0.6930 - accuracy: 0.5120
Epoch 9/120
3/3 [==============================] - 0s 2ms/step - loss: 0.6955 - accuracy: 0.5092
Epoch 10/120
3/3 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5074
Epoch 11/120
3/3 [===========

In [42]:
# Evaluate model on the test data
model.evaluate(X1_test,y_test, verbose=2)

23/23 - 0s - loss: 0.6935 - accuracy: 0.5311 - 36ms/epoch - 2ms/step


[0.6935397982597351, 0.5311203598976135]

In [43]:
# Save predictions on the test data
predictions = model.predict(X1_test)
predictions

23/23 [==============================] - 0s 863us/step


array([[0.5107974 , 0.4892026 ],
       [0.47308025, 0.5269198 ],
       [0.5160056 , 0.48399445],
       ...,
       [0.48773307, 0.5122669 ],
       [0.51084787, 0.48915216],
       [0.49533987, 0.5046601 ]], dtype=float32)

In [44]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
# Convert most likely category back to original labels
most_likely = converter.inverse_transform((most_likely))
most_likely[:10]

array(['buy', 'sell', 'buy', 'buy', 'sell', 'sell', 'sell', 'buy', 'buy',
       'sell'], dtype=object)

In [45]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

buy     391
sell    332
dtype: int64

## Use Bollinger Bands indicator to visualize upper and lower price range levels

In [23]:
# Create a new clean dataframe to filter price, SMA_Short and SMA_Long only
ltc_yf['SMA20']= ltc_df[['SMA_Short']].copy()
ltc_yf['SMA100']= ltc_df[['SMA_Long']].copy()
ltc_yf['Trade Type'] = ltc_df[['Trade_Type']].copy()
display(ltc_yf.tail())

,Open,High,Low,Close,Volume,SMA20,SMA100,Trade Type
Date,,,,,,,,
2022-11-23,70.464264,81.563438,69.785896,79.000069,2115042339,62.694070,56.736650,sell
2022-11-24,78.999107,79.031052,76.008377,78.830147,1248286495,63.255748,56.911528,buy
2022-11-25,78.807121,78.857140,73.604179,74.028900,891650503,63.471133,57.048075,buy
2022-11-26,74.021461,79.247986,74.021461,76.372894,765406705,63.885475,57.211347,sell
2022-11-27,76.383530,78.890190,74.849884,75.044319,622191865,64.255077,57.421474,buy


In [24]:
# Get Bollinger Bands
bb_df = TA.BBANDS(ltc_yf)
bb_df.tail()

,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,
2022-11-23,75.333005,62.694070,50.055135
2022-11-24,77.683837,63.255748,48.827659
2022-11-25,78.424655,63.471133,48.517610
2022-11-26,79.805417,63.885475,47.965533
2022-11-27,80.871243,64.255077,47.638911


In [27]:
# Combine two dataframe together
bb_ltc = pd.concat([ltc_yf, bb_df], axis=1)
bb_ltc = bb_ltc.dropna()
display(bb_ltc.head())
display(bb_ltc.tail())

,Open,High,Low,Close,Volume,SMA20,SMA100,Trade Type,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,,,
2014-12-26,2.70070,2.81018,2.68167,2.78169,2088360,3.157594,3.737192,buy,3.869390,3.157594,2.445798
2014-12-27,2.77958,2.79537,2.70296,2.71695,2180800,3.107883,3.717509,buy,3.795353,3.107883,2.420414
2014-12-28,2.71900,2.76183,2.67281,2.73304,1701950,3.061875,3.701562,sell,3.718136,3.061875,2.405615
2014-12-29,2.73380,2.74722,2.68709,2.69482,1471530,3.020558,3.685645,buy,3.658871,3.020558,2.382245
2014-12-30,2.69617,2.72399,2.66743,2.70401,1536350,2.980765,3.670226,sell,3.591918,2.980765,2.369613


,Open,High,Low,Close,Volume,SMA20,SMA100,Trade Type,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,,,
2022-11-23,70.464264,81.563438,69.785896,79.000069,2115042339,62.694070,56.736650,sell,75.333005,62.694070,50.055135
2022-11-24,78.999107,79.031052,76.008377,78.830147,1248286495,63.255748,56.911528,buy,77.683837,63.255748,48.827659
2022-11-25,78.807121,78.857140,73.604179,74.028900,891650503,63.471133,57.048075,buy,78.424655,63.471133,48.517610
2022-11-26,74.021461,79.247986,74.021461,76.372894,765406705,63.885475,57.211347,sell,79.805417,63.885475,47.965533
2022-11-27,76.383530,78.890190,74.849884,75.044319,622191865,64.255077,57.421474,buy,80.871243,64.255077,47.638911


In [30]:
# Cut the date range from 2017 to 2022
bb_ltc = bb_ltc.loc['2017-01-01':'2022-11-27']
display(bb_ltc.head())
display(bb_ltc.tail())

,Open,High,Low,Close,Volume,SMA20,SMA100,Trade Type,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,,,
2017-01-01,4.32912,4.52153,4.32871,4.51159,11337500,4.065948,3.903264,sell,4.917335,4.065948,3.214560
2017-01-02,4.52045,4.73766,4.48466,4.64894,14595300,4.115881,3.911702,sell,4.981641,4.115881,3.250120
2017-01-03,4.65588,4.73184,4.56487,4.62620,10055700,4.166213,3.919881,buy,5.027515,4.166213,3.304910
2017-01-04,4.63375,4.83785,4.63375,4.83785,13762400,4.226882,3.929672,sell,5.098389,4.226882,3.355374
2017-01-05,4.91437,4.94586,4.22687,4.29272,19196000,4.260109,3.934077,buy,5.084929,4.260109,3.435289


,Open,High,Low,Close,Volume,SMA20,SMA100,Trade Type,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,,,
2022-11-23,70.464264,81.563438,69.785896,79.000069,2115042339,62.694070,56.736650,sell,75.333005,62.694070,50.055135
2022-11-24,78.999107,79.031052,76.008377,78.830147,1248286495,63.255748,56.911528,buy,77.683837,63.255748,48.827659
2022-11-25,78.807121,78.857140,73.604179,74.028900,891650503,63.471133,57.048075,buy,78.424655,63.471133,48.517610
2022-11-26,74.021461,79.247986,74.021461,76.372894,765406705,63.885475,57.211347,sell,79.805417,63.885475,47.965533
2022-11-27,76.383530,78.890190,74.849884,75.044319,622191865,64.255077,57.421474,buy,80.871243,64.255077,47.638911


In [47]:
# Plot Bollinger Bands and Closed price in a graph
ltc_close = bb_ltc[['Close']].hvplot(
    line_color = 'grey',
    ylabel='Price in $',
    width=1000,
    height=400)

bb_upper = bb_ltc[['BB_UPPER']].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400)

bb_middle = bb_ltc[['BB_MIDDLE']].hvplot(
    line_color='skyblue',
    ylabel='Price in $',
    width=1000,
    height=400)

bb_lower = bb_ltc[['BB_LOWER']].hvplot(
    line_color = 'orange',
    ylabel='Price in $',
    width=1000,
    height=400)

# Overlay plots
bbands_plot = ltc_close * bb_upper * bb_middle * bb_lower
bbands_plot.opts(opts.Overlay(title='Litecoin Closed price with Bollinger Bands'))

:Overlay
   .Curve.I   :Curve   [Date]   (Close)
   .Curve.II  :Curve   [Date]   (BB_UPPER)
   .Curve.III :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV  :Curve   [Date]   (BB_LOWER)

In [46]:
# Plot SMA and closed price in one graph
bb_sma20 = bb_ltc[['SMA20']].hvplot(
    line_color = 'green',
    ylabel='Price in $',
    width=1000,
    height=400)

bb_sma100 = bb_ltc[['SMA100']].hvplot(
    line_color = 'pink',
    ylabel='Price in $',
    width=1000,
    height=400)

# Overlay plots
sma_plot = ltc_close * bb_sma20 * bb_sma100
sma_plot.opts(opts.Overlay(title='Litecoin Closed Price with SMA 20 Days vs 100 Days'))

:Overlay
   .Curve.I   :Curve   [Date]   (Close)
   .Curve.II  :Curve   [Date]   (SMA20)
   .Curve.III :Curve   [Date]   (SMA100)

In [48]:
bbands_plot + sma_plot

:Layout
   .Overlay.I  :Overlay
      .Curve.I   :Curve   [Date]   (Close)
      .Curve.II  :Curve   [Date]   (BB_UPPER)
      .Curve.III :Curve   [Date]   (BB_MIDDLE)
      .Curve.IV  :Curve   [Date]   (BB_LOWER)
   .Overlay.II :Overlay
      .Curve.I   :Curve   [Date]   (Close)
      .Curve.II  :Curve   [Date]   (SMA20)
      .Curve.III :Curve   [Date]   (SMA100)